In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np


# In[2]:


class RankNet(nn.Module):
    def __init__(self, num_feature):
        super(RankNet, self).__init__()

        self.model = nn.Sequential(
            nn.Linear( num_feature, 512),
            nn.Dropout(0.5),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.Dropout(0.5),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )
        self.output_sig = nn.Sigmoid()

    def forward(self, input_1,input_2):


        s1 = self.model(input_1)
        s2 = self.model(input_2)
        
        out = self.output_sig(s1-s2)

        return out
    
    def predict(self, input_):
        
        s = self.model(input_)
        return s


# In[12]:


n_sample = 30
n_feature = 30
data1 = torch.rand((n_sample,n_feature))
data2 = torch.rand((n_sample, n_feature))

y = np.random.random(( n_sample,1))
y = y > 0.9
y = 1. * y
y = torch.Tensor(y)

rank_model = RankNet(num_feature=n_feature)
optimizer = torch.optim.Adam(rank_model.parameters())

loss_fun = torch.nn.BCELoss()

rank_model.cuda()
# optimizer.cuda()
loss_fun.cuda()

data1 = data1.cuda()
data2 = data2.cuda()

y = y.cuda()




In [10]:
epoch = 20000

losses = []

for i in range(epoch):
    
    rank_model.zero_grad()
    
    y_pred = rank_model(data1, data2)
    
    loss = loss_fun(y_pred,y)
#     print('y: {} {}'.format(y.shape, y))
#     print('y_pred: {} {}'.format(y_pred.shape, y_pred))
    
#     if i == 2:
#         raise Exception()
    
    loss.backward()
    optimizer.step()
    
    losses.append(loss.item())
    
    if i % 500 == 0:
    
        print('Epoch{}, loss : {}'.format(i, loss.item()))
    



y: torch.Size([30, 1]) tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.]], device='cuda:0')
y_pred: torch.Size([30, 1]) tensor([[0.4964],
        [0.5194],
        [0.5135],
        [0.4922],
        [0.5095],
        [0.4884],
        [0.5092],
        [0.4945],
        [0.5026],
        [0.5009],
        [0.4903],
        [0.4914],
        [0.4822],
        [0.4919],
        [0.4927],
        [0.5017],
        [0.5139],
        [0.4955],
        [0.4850],
        [0.4886],
        [0.5170],
        [0.5068],
        [0.5008],
        [0.5055],
        [0.4927],
        [0.4960],
        [0.4942],
        [0.5043],
       

Exception: 

In [ ]:
import matplotlib.pyplot as plt
x = list(range(len(losses)))
plt.plot(x, losses)